In [2]:
!pip install --upgrade pinecone

In [3]:
!pip install --upgrade --quiet pinecone-client openai langchain pypdf sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 765.0/765.0 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.7/309.7 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 470.2/470.2 kB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 58.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 48.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [10]:
!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 33.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.3 MB/s eta 0:00:00


In [4]:
!pip install PyPDF2

In [11]:
import fitz  # PyMuPDF
import os
import openai
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Pinecone
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains.question_answering import load_qa_chain

In [12]:
import pdfplumber

def load_pdf(path):
    text = ""
    with pdfplumber.open(path) as pdf:
        for page in pdf.pages:
            text += page.extract_text() + "\n"
    return text

pdf_text = load_pdf("/budget_speech.pdf")


In [16]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = splitter.split_text(pdf_text)


In [24]:
!pip install pinecone-notebooks

In [25]:
if not os.environ.get("pcsk_65KdEB_3qYK83nZKETac9MTrVM2Sgtyb9Jw3HSRfG7FPJGZnbZ38TVTX1uXMYJTj2fTLk1"):
  from pinecone_notebooks.colab import Authenticate
  Authenticate()

In [27]:
from pinecone import Pinecone

# Initialize a Pinecone client with your API key
api_key = os.environ.get("pcsk_65KdEB_3qYK83nZKETac9MTrVM2Sgtyb9Jw3HSRfG7FPJGZnbZ38TVTX1uXMYJTj2fTLk1")
pc = Pinecone(api_key=api_key)

In [37]:
from pinecone import Pinecone, ServerlessSpec

# Initialize Pinecone client
pc = Pinecone(
    api_key=os.environ.get("pcsk_65KdEB_3qYK83nZKETac9MTrVM2Sgtyb9Jw3HSRfG7FPJGZnbZ38TVTX1uXMYJTj2fTLk1")
)

if 'budget-speech-quiz' not in pc.list_indexes().names():
      pc.create_index(
          name='budget-speech-quiz',
          dimension=384,
          metric='cosine',
          spec=ServerlessSpec(
              cloud='aws',
              region='us-east-1'
          )
      )

In [38]:
!pip install sentence-transformers

from sentence_transformers import SentenceTransformer

model = SentenceTransformer("all-MiniLM-L6-v2")

In [39]:
from tqdm import tqdm

index = pc.Index("budget-speech-quiz")
for i, chunk in enumerate(tqdm(chunks)):
    embedding = model.encode(chunk)
    index.upsert([(f"chunk-{i}", embedding, {"text": chunk})])


100%|██████████| 189/189 [00:48<00:00,  3.91it/s]


In [44]:
!pip install transformers torch


In [45]:
from transformers import pipeline

qa_pipeline = pipeline("question-answering", model="distilbert-base-cased-distilled-squad")


config.json:   0%|          | 0.00/473 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/261M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

Device set to use cpu


In [46]:
def ask_question(question):
    # Get embedding and search Pinecone
    question_embedding = model.encode(question).tolist()
    results = index.query(vector=question_embedding, top_k=3, include_metadata=True)

    # Combine retrieved texts as context
    context = "\n\n".join([match["metadata"]["text"] for match in results["matches"]])

    # Use local QA model to answer
    result = qa_pipeline(question=question, context=context)

    return result["answer"]


In [52]:
question = "Sabka Saath Sabka Vikas sustained focus on?"
answer = ask_question(question)
print("Answer:", answer)


Answer: Jammu &
Kashmir, Ladakh and the North-East


In [53]:
import nbformat

file_path = "/content/budget-speech-quiz.ipynb"

with open(file_path, "r", encoding="utf-8") as f:
    nb = nbformat.read(f, as_version=4)

# Clean the widgets metadata
if "widgets" in nb.metadata:
    del nb.metadata["widgets"]

with open(file_path, "w", encoding="utf-8") as f:
    nbformat.write(nb, f)

print("Cleaned metadata.widgets from notebook.")


Cleaned metadata.widgets from notebook.


In [54]:
!pip install gradio --quiet


In [55]:
import gradio as gr

# Wrap your ask_question function with Gradio interface
iface = gr.Interface(
    fn=ask_question,           # Your function that takes a question and returns an answer
    inputs=gr.Textbox(lines=2, placeholder="Type your question here..."),  # Input widget
    outputs="text",            # Output will be plain text
    title="PDF Budget Speech Quiz",
    description="Ask any question about the budget speech PDF and get answers."
)

# Launch the Gradio app (runs in Colab and gives a public URL)
iface.launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://1509324cf6b9cc6452.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [56]:
!pip install transformers gradio


In [57]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
import random

qg_model = T5ForConditionalGeneration.from_pretrained("iarfmoose/t5-base-question-generator")
qg_tokenizer = T5Tokenizer.from_pretrained("iarfmoose/t5-base-question-generator")


config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/892M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/121 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [58]:
def generate_mcq(context):
    # Generate question
    input_text = f"generate question: {context.strip()}"
    input_ids = qg_tokenizer.encode(input_text, return_tensors="pt")
    output_ids = qg_model.generate(input_ids, max_length=64, num_beams=4, early_stopping=True)
    question = qg_tokenizer.decode(output_ids[0], skip_special_tokens=True)

    # Correct answer is part of context (simplified assumption)
    correct_answer = context.strip().split()[-1]

    # Generate distractors (dummy for now; we can improve later)
    distractors = random.sample([
        "growth", "inflation", "revenue", "subsidy", "exports", "budget", "loan", "tax"
    ], 3)

    options = distractors + [correct_answer]
    random.shuffle(options)

    return {
        "question": question,
        "options": options,
        "answer": correct_answer
    }


In [59]:
def get_random_chunk():
    # You can also sample intelligently
    results = index.query(vector=model.encode("generate quiz").tolist(), top_k=10, include_metadata=True)
    chunk = random.choice(results["matches"])["metadata"]["text"]
    return chunk


In [60]:
import gradio as gr

def generate_quiz():
    chunk = get_random_chunk()
    mcq = generate_mcq(chunk)
    return mcq["question"], mcq["options"], mcq["answer"]

iface = gr.Interface(
    fn=generate_quiz,
    inputs=[],
    outputs=[
        gr.Textbox(label="Question"),
        gr.List(label="Options"),
        gr.Textbox(label="Correct Answer")
    ],
    title="Budget Speech MCQ Generator",
    description="Generates a multiple-choice question from the budget speech PDF."
)

iface.launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://1b8515609f2946589f.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
